#### If you are working on a new environmnet, you might want to run the following cell to make sure that the MIDIUtil library has been installed 

In [ ]:
!pip install midiutil

#### Import the variables and definitions from the accompanying music_def.py file 

In [ ]:
# This file contains all the definitions for the widgets and some constant variables that I had set

from music_def import *

***

### Define your key, scale, and octave

In [ ]:
# First choice with regards to music

# Run this cell to get the widgets to show
display(vert_note)
display(vert_scale)
display(vert_octave)

In [ ]:
# Processing inputs from cell above

note_index = notes_to_index[def_note.value]
scale_num = scale_index[def_scale.value]
scale_clean = all_scales[scale_num]

# Printing outputs to allow user to check whether they are happy with their choices

print(
    "--------------------------------------------------------------------------------------------------------------------------"
)

print(
    "You have chosen the Key,",
    def_note.value,
    "which corresponds to the index: \n",
    note_index,
)
print(
    "The chosen scale is a ",
    def_scale.value,
    "which corresponds to the midi indices: \n",
    scale_clean,
)
print("The octave that you have chose to map your notes onto is: \n ", def_octave.value)

print(
    "If this is incorrect, please check with the drop down menu above and run this cell again."
)

# Mapping chosen scale onto each other
scale = []
for ix in scale_clean:
    a = ix + (len_oct * def_octave.value) + note_index
    scale.append(a)

print(
    "--------------------------------------------------------------------------------------------------------------------------"
)

***

### Looking at datafile

##### The variable "file" is for the whole data file
##### The variable "data" is used for the part that we want to convert to music 

In [ ]:
# For now we can take the noisy C60 IR as the guinea pig again
# This part will differ depending on your data

# Loading file called "C60plus.csv" and saving the values in the first column as a list called "data "

file = np.loadtxt("sample_data/C60plus.csv", skiprows=1, delimiter=",")
data = file[:, 1]

# First look at the data by plotting the first column against the second
plt.plot(file[:, 0], file[:, 1])

### If I want to zoom in or out?
##### Note: the more you zoom the more features your resulting sound will have depending on the noise of your data

In [ ]:
# Selecting Data range
# Zoom widget, this cannot be predefined as it must be redefined after data gets assigned to your data
zoom_range = widgets.FloatRangeSlider(
    min=np.min(data),
    max=np.max(data),
    value=[np.min(data), np.max(data)],
    step=0.01,
    description="Zoom Range:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
display(zoom_range)

#### Use the zoom widget in the cell above to chose your range and then run the cell below to visualise the data once the zoom has been applied 
#### You need to re-run the cell below everytime you change the range above! 

In [ ]:
# Look at data again with zoomed values on y axis

# Assigning the min and max based on the widget value above

low = zoom_range.value[0]
high = zoom_range.value[1]

# Or if you wish to manually define the low and high points
# low =
# high =

# Plotting with the zoom
plt.plot(file[:, 0], file[:, 1])
plt.ylim(low, high)
plt.show()

# Once you are happy with the range, we bin the data to match our chosen scale

bins = np.linspace(
    low, high, len(scale)
)  # Defining bins based of the max, min, and length of a scale
abs_binned = np.digitize(data, bins)  # sorting the data into the bins defined above
unique_bins = np.unique(abs_binned)  # making an array of our unique bins

revalue = dict(
    zip(unique_bins, scale)
)  # dictionary to map our scale to the binned values of our IR absorption

## ALTERNATIVE
# # split the scale based on how many unique bins there are
# revalue = dict(zip(unique_bins, scale[::int(len(scale)/len(unique_bins))]))

notes = [
    revalue.get(n, n) for n in abs_binned
]  # mapping the data values onto the scale

****

### Definine rest of parameters for MIDIUtil to work

#### If you are confused on the choices please visit the MIDIUtil Docs, a link to it can be found on the learn page. 

In [ ]:
# Displaying the pre-defined widgets to help with musical choice for writing your MIDI file

display(vert_channel)
display(vert_tempo)
display(vert_volume)
print("About time signature")
display(vert_bpb)
display(vert_type_beat)

In [ ]:
# Set up some variables for the MIDIUtil program to work

# This takes your inputs from above and converts it into a format that the MIDIUtil library can read

channel = int(def_channel.value)
tempo = int(def_tempo.value)
volume = int(def_volume.value)

# Final inputs for wirting the MIDI file

display(vert_name)
display(vert_int)

In [ ]:
# Writing a MIDI file based on your choices

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
# automatically)
MyMIDI.addTempo(track, time, tempo)
MyMIDI.addTimeSignature(
    track,
    time,
    def_bpb.value,
    note_name_dictionary[def_type_beat.value],
    clocks_per_tick,
    notes_per_quarter,
)  # add a time signature, here just 4/4

# loop over each time step

i = 0
for n in notes[::4]:  # loop over each note
    MyMIDI.addNote(track, channel, n, time + i, duration, volume)
    i = i + 1

# saving the resulting midi file.

with open(str(def_name.value) + ".mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)